### Prof. Pedram Jahangiry

You need to make a copy to your own Google drive if you want to edit the original notebook! Start by opening this notebook on Colab 👇

<a href="https://colab.research.google.com/github/PJalgotrader/Deep_forecasting-USU/blob/main/Lectures%20and%20codes/DF%20Spring%202023/Module%204-%20ARIMA/Module4-ARIMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a> 



![logo](https://upload.wikimedia.org/wikipedia/commons/4/44/Huntsman-Wordmark-with-USU-Blue.gif#center) 


## 🔗 Links

[![linkedin](https://img.shields.io/badge/LinkedIn-0A66C2?style=for-the-badge&logo=linkedin&logoColor=white)](https://www.linkedin.com/in/pedram-jahangiry-cfa-5778015a)

[![Youtube](https://img.shields.io/badge/youtube_channel-1DA1F2?style=for-the-badge&logo=youtube&logoColor=white&color=FF0000)](https://www.youtube.com/channel/UCNDElcuuyX-2pSatVBDpJJQ)

[![Twitter URL](https://img.shields.io/twitter/url/https/twitter.com/PedramJahangiry.svg?style=social&label=Follow%20%40PedramJahangiry)](https://twitter.com/PedramJahangiry)


---


# Module 4: ARIMA models

In this module, we cover the basics of ARIMA (AutoRegressive Integrated Moving Average) models, a commonly used statistical method for time series forecasting. Our focus will be on understanding the underlying concepts and components of ARIMA models, as well as how to implement them in practice.

We start by discussing the properties of time series data and the need for a statistical model to capture its behavior. Next, we delve into the components of ARIMA models - autoregression, integration, and moving average - and their role in capturing patterns and making predictions based on past values.

We also cover the process of making time series data stationary and selecting the appropriate ARIMA parameters (p, d, q) based on autocorrelation and partial autocorrelation plots. Finally, we demonstrate how to fit ARIMA models to time series data and make predictions using Python packages such as sktime and PyCaret.

Documentation: 

1. **PyCaret**: https://pycaret.readthedocs.io/en/latest/index.html PyCaret3.0
2. **sktime** : https://www.sktime.org/en/stable/api_reference/forecasting.html

# Installation

Follow the steps here: https://pycaret.gitbook.io/docs/get-started/installation


In [1]:
#only if you want to run it in Google Colab: 
# for this chapter, we can install the light version of PyCaret as below. 

!pip install --pre pycaret

In [2]:
# if you got a warning that you need to "RESTART RUNTIME", go ahead and press that button. 

# let's double ckeck the Pycaret version: 
from pycaret.utils import version
version()

'3.0.0.rc4'

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Importing Dataset

In [4]:
from pycaret.datasets import get_data
airline = get_data('airline')

Period
1949-01    112.0
1949-02    118.0
1949-03    132.0
1949-04    129.0
1949-05    121.0
Freq: M, Name: Number of airline passengers, dtype: float64

In [5]:
# or alternatively, 
df = pd.read_csv("https://raw.githubusercontent.com/PJalgotrader/Deep_forecasting-USU/main/data/airline_passengers.csv", index_col="Month")
df.head()

,Passengers
Month,
1949-01,112
1949-02,118
1949-03,132
1949-04,129
1949-05,121


In [6]:
# if you are working with Pandas, your first job should be changing the type of the index to datetime and then to period! This is a compatibility issue with other packages. 
df.index

Index(['1949-01', '1949-02', '1949-03', '1949-04', '1949-05', '1949-06',
       '1949-07', '1949-08', '1949-09', '1949-10',
       ...
       '1960-03', '1960-04', '1960-05', '1960-06', '1960-07', '1960-08',
       '1960-09', '1960-10', '1960-11', '1960-12'],
      dtype='object', name='Month', length=144)

In [7]:
df.index = pd.to_datetime(df.index).to_period('M')
df.index

PeriodIndex(['1949-01', '1949-02', '1949-03', '1949-04', '1949-05', '1949-06',
             '1949-07', '1949-08', '1949-09', '1949-10',
             ...
             '1960-03', '1960-04', '1960-05', '1960-06', '1960-07', '1960-08',
             '1960-09', '1960-10', '1960-11', '1960-12'],
            dtype='period[M]', name='Month', length=144)

Setting up PyCaret Experiment:

In [8]:
from pycaret.time_series import *

In [9]:
exp = TSForecastingExperiment()
exp.setup(data = df, target='Passengers' ,  fh = 12, coverage=0.95)

,Description,Value
0,session_id,4403
1,Target,Passengers
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(144, 1)"
5,Transformed data shape,"(144, 1)"
6,Transformed train set shape,"(132, 1)"
7,Transformed test set shape,"(12, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [75]:
exp.check_stats()

,Test,Test Name,Data,Property,Setting,Value
0,Summary,Statistics,Transformed,Length,,144.0
1,Summary,Statistics,Transformed,# Missing Values,,0.0
2,Summary,Statistics,Transformed,Mean,,280.298611
3,Summary,Statistics,Transformed,Median,,265.5
4,Summary,Statistics,Transformed,Standard Deviation,,119.966317
5,Summary,Statistics,Transformed,Variance,,14391.917201
6,Summary,Statistics,Transformed,Kurtosis,,-0.364942
7,Summary,Statistics,Transformed,Skewness,,0.58316
8,Summary,Statistics,Transformed,# Distinct Values,,118.0
9,White Noise,Ljung-Box,Transformed,Test Statictic,"{'alpha': 0.05, 'K': 24}",1606.083817


In [10]:
exp.plot_model(plot='train_test_split')

---
---
# ARIMA models:

## Selecting p and q: 
Remember, to select p and q, we must first make the dat astationary! That's why we will plot the difference model. 

**Difference plotting using orders:**

In [14]:
exp.plot_model(plot="diff", data_kwargs={"order_list": [1,2], "acf": True, "pacf": True})


**Difference Plot Using Lags:**

For example, given a timeseries with monthly periodicity, using lags=[1, 12] corresponds to applying a standard first difference to handle trend, and followed by a seasonal difference (at lag 12) to attempt to account for seasonal dependence.

In [15]:
exp.plot_model(plot="diff", data_kwargs={"lags_list": [[1, 12]], "acf": True, "pacf": True})


Based on the above plot, it seems that SARIMA(1,1,1)(0,1,0,12) is a good start. 

However, to compare the performance of different components of ARIMA model, let's construct 4 more models. So we have 5 models to compare + two bench marks! 
1. AR(1)
2. MA(1)
3. ARMA(1,1)
4. ARIMA(1,1,1)
5. SARIMA(1,1,1)(0,1,0,12)
6. Random walk: ARIMA(0,1,0) with no constant
7. Random walk with drift: ARIMA(0,1,0) with constant

---
### ARIMA

In [92]:
ar1 = exp.create_model('arima', order = (1,1,0), seasonal_order=(0,0,0,12), with_intercept=True, cross_validation=False) 
# by default, "with_intercept=True", we don't need to add it mannually. 

,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
Test,1.9700,2.4082,59.9858,83.1988,0.1140,0.1235,-0.2496


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [82]:
ma1 = exp.create_model('arima', order = (0,1,1), seasonal_order=(0,0,0,12), with_intercept= True, cross_validation=False)

,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
Test,1.9635,2.3544,59.7871,81.3420,0.1148,0.1232,-0.1944


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [83]:
arma11 = exp.create_model('arima', order = (1,1,1), seasonal_order=(0,0,0,12), with_intercept= True, cross_validation=False)

,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
Test,1.9990,2.3844,60.8693,82.3772,0.1166,0.1256,-0.2250


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [84]:
arima111= exp.create_model('arima', order = (1,1,1), seasonal_order=(0,0,0,12) , with_intercept= True, cross_validation=False)

,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
Test,1.9990,2.3844,60.8693,82.3772,0.1166,0.1256,-0.2250


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [85]:
sarima111010= exp.create_model('arima', order = (1,1,1), seasonal_order=(0,1,0,12) , with_intercept= True, cross_validation=False)

,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
Test,0.7354,0.8041,22.3941,27.7807,0.0505,0.0485,0.8607


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [100]:
rw= exp.create_model('arima', order = (0,1,0), seasonal_order=(0,0,0,12) , with_intercept= False, cross_validation=False)
# remember, Random walk is equivalent to naive forecaster. So this code also works: exp.create_model('naive', cross_validation=False)

,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
Test,2.4959,2.9807,76.0000,102.9765,0.1425,0.1612,-0.9143


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [101]:
rwwd= exp.create_model('arima', order = (0,1,0), seasonal_order=(0,0,0,12) , with_intercept= True, cross_validation=False)

,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
Test,2.1776,2.6822,66.3079,92.6664,0.1242,0.1381,-0.5502


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [109]:
my_models = [rw, rwwd, ar1, ma1, arma11, arima111, sarima111010]
my_model_lables = ['Random Walk', 'Random Walk with drift', 'AR(1)', 'MA(1)', 'ARMA(1,1)', 'ARIMA(1,1,1)', 'SARIMA(1,1,1)(0,1,0,12)']

In [111]:
exp.compare_models(my_models, cross_validation=False)

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
6,ARIMA,0.7354,0.8041,22.3941,27.7807,0.0505,0.0485,0.8607,0.2800
3,ARIMA,1.9635,2.3544,59.7871,81.3420,0.1148,0.1232,-0.1944,0.1400
2,ARIMA,1.9700,2.4082,59.9858,83.1988,0.1140,0.1235,-0.2496,0.0800
4,ARIMA,1.9990,2.3844,60.8693,82.3772,0.1166,0.1256,-0.2250,0.2100
5,ARIMA,1.9990,2.3844,60.8693,82.3772,0.1166,0.1256,-0.2250,0.2100
1,ARIMA,2.1776,2.6822,66.3079,92.6664,0.1242,0.1381,-0.5502,0.1100
0,ARIMA,2.4959,2.9807,76.0000,102.9765,0.1425,0.1612,-0.9143,0.0800


Processing:   0%|          | 0/33 [00:00<?, ?it/s]

ARIMA(order=(1, 1, 1), seasonal_order=(0, 1, 0, 12))

---
**So the winner is SARIMA(1,1,1)(0,1,0,12)**

In [104]:
sarima111010

ARIMA(order=(1, 1, 1), seasonal_order=(0, 1, 0, 12))

In [105]:
sarima111010.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                      
===========================================================================================
Dep. Variable:                                   y   No. Observations:                  132
Model:             SARIMAX(1, 1, 1)x(0, 1, [], 12)   Log Likelihood                -447.278
Date:                             Thu, 02 Feb 2023   AIC                            902.557
Time:                                     18:48:32   BIC                            913.673
Sample:                                 01-31-1949   HQIC                           907.071
                                      - 12-31-1959                                         
Covariance Type:                               opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.8162      1.277      0.639      0.523      -1.687       3.320
ar.L1         -0.5518      0.305     -1.806      0.071      -1.150       0.047
ma.L1          0.3291      0.333      0.987      0.324      -0.324       0.982
sigma2       107.6414     12.948      8.313      0.000      82.263     133.020
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):                 1.30
Prob(Q):                              1.00   Prob(JB):                         0.52
Heteroskedasticity (H):               1.49   Skew:                            -0.08
Prob(H) (two-sided):                  0.21   Kurtosis:                         3.49
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

**Exercise**: write down the equation for the SARIMA model?

---
#### Plotting models

In [138]:
exp.plot_model(sarima111010  , plot='forecast', data_kwargs={'fh':20, 'labels':['SARIMA(1,1,1)(0,1,0,12)']})

In [139]:
exp.plot_model(sarima111010, plot='diagnostics')

In [121]:
# let's test the stationarity of the residuals for the SARIMA(1,1,1)(0,1,0,12) model:
exp.check_stats(sarima111010, test = 'adf')


,Test,Test Name,Data,Property,Setting,Value
0,Stationarity,ADF,Residual,Stationarity,{'alpha': 0.05},True
1,Stationarity,ADF,Residual,p-value,{'alpha': 0.05},0.0
2,Stationarity,ADF,Residual,Test Statistic,{'alpha': 0.05},-11.694324
3,Stationarity,ADF,Residual,Critical Value 1%,{'alpha': 0.05},-3.481682
4,Stationarity,ADF,Residual,Critical Value 5%,{'alpha': 0.05},-2.884042
5,Stationarity,ADF,Residual,Critical Value 10%,{'alpha': 0.05},-2.57877


In [113]:
my_models

[ARIMA(order=(0, 1, 0), seasonal_order=(0, 0, 0, 12), with_intercept=False),
 ARIMA(order=(0, 1, 0), seasonal_order=(0, 0, 0, 12)),
 ARIMA(order=(1, 1, 0), seasonal_order=(0, 0, 0, 12)),
 ARIMA(order=(0, 1, 1), seasonal_order=(0, 0, 0, 12)),
 ARIMA(order=(1, 1, 1), seasonal_order=(0, 0, 0, 12)),
 ARIMA(order=(1, 1, 1), seasonal_order=(0, 0, 0, 12)),
 ARIMA(order=(1, 1, 1), seasonal_order=(0, 1, 0, 12))]

In [112]:
my_model_lables

['Random Walk',
 'Random Walk with drift',
 'AR(1)',
 'MA(1)',
 'ARMA(1,1)',
 'ARIMA(1,1,1)',
 'SARIMA(1,1,1)(0,1,0,12)']

In [115]:
exp.plot_model(my_models, plot='forecast', data_kwargs={'fh':36, 'labels':my_model_lables})

In [117]:
exp.plot_model(my_models, plot='insample', data_kwargs={'labels':my_model_lables})

---
### Auto ARIMA
https://www.sktime.org/en/stable/api_reference/auto_generated/sktime.forecasting.arima.AutoARIMA.html

Wrapper of the pmdarima implementation of fitting Auto-(S)ARIMA(X) models. The auto-ARIMA algorithm seeks to identify the most optimal parameters for an ARIMA model, settling on a single fitted ARIMA model. This process is based on the commonly-used R function, forecast::auto.arima.

Auto-ARIMA works by conducting differencing tests (i.e., Kwiatkowski–Phillips–Schmidt–Shin, Augmented Dickey-Fuller or Phillips–Perron) to determine the order of differencing, d, and then fitting models within ranges of defined start_p, max_p, start_q, max_q ranges. If the seasonal optional is enabled, auto-ARIMA also seeks to identify the optimal P and Q hyper-parameters after conducting the Canova-Hansen to determine the optimal order of seasonal differencing, D.

In order to find the best model, auto-ARIMA optimizes for a given information_criterion, one of (‘aic’, ‘aicc’, ‘bic’, ‘hqic’, ‘oob’) (Akaike Information Criterion, Corrected Akaike Information Criterion, Bayesian Information Criterion, Hannan-Quinn Information Criterion, or “out of bag”–for validation scoring–respectively) and returns the ARIMA which minimizes the value


In [122]:
auto_arima = exp.create_model('auto_arima', cross_validation=False)

,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
Test,0.4893,0.5365,14.8982,18.5365,0.0310,0.0309,0.9380


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [123]:
auto_arima.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                     SARIMAX Results                                      
==========================================================================================
Dep. Variable:                                  y   No. Observations:                  132
Model:             SARIMAX(3, 0, 0)x(0, 1, 0, 12)   Log Likelihood                -447.843
Date:                            Thu, 02 Feb 2023   AIC                            905.686
Time:                                    19:13:37   BIC                            919.623
Sample:                                01-31-1949   HQIC                           911.346
                                     - 12-31-1959                                         
Covariance Type:                              opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      5.5341      2.007      2.757      0.006       1.600       9.468
ar.L1          0.7049      0.095      7.393      0.000       0.518       0.892
ar.L2          0.2574      0.131      1.968      0.049       0.001       0.514
ar.L3         -0.1434      0.107     -1.338      0.181      -0.354       0.067
sigma2       101.0969     12.818      7.887      0.000      75.974     126.220
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):                 2.83
Prob(Q):                              0.96   Prob(JB):                         0.24
Heteroskedasticity (H):               1.41   Skew:                            -0.14
Prob(H) (two-sided):                  0.29   Kurtosis:                         3.70
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [125]:
# recall, 
sarima111010.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                      
===========================================================================================
Dep. Variable:                                   y   No. Observations:                  132
Model:             SARIMAX(1, 1, 1)x(0, 1, [], 12)   Log Likelihood                -447.278
Date:                             Thu, 02 Feb 2023   AIC                            902.557
Time:                                     19:14:57   BIC                            913.673
Sample:                                 01-31-1949   HQIC                           907.071
                                      - 12-31-1959                                         
Covariance Type:                               opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.8162      1.277      0.639      0.523      -1.687       3.320
ar.L1         -0.5518      0.305     -1.806      0.071      -1.150       0.047
ma.L1          0.3291      0.333      0.987      0.324      -0.324       0.982
sigma2       107.6414     12.948      8.313      0.000      82.263     133.020
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):                 1.30
Prob(Q):                              1.00   Prob(JB):                         0.52
Heteroskedasticity (H):               1.49   Skew:                            -0.08
Prob(H) (two-sided):                  0.21   Kurtosis:                         3.49
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

Comparing the output of our own SARIMA and auto ARIMA, our sarima111010 generates better AIC (smaller) number in the trainset. However, based on R2, the auto_arima is a better model in the test set. We continue to work with our own SARIMA model because we understand it better :)

In [129]:
exp.plot_model([sarima111010, auto_arima], plot='forecast')

## In-sample performance metrics? 

In [130]:
# recall, our forecasting horizon was 12 months.
df.index[:-12] # train set index 

PeriodIndex(['1949-01', '1949-02', '1949-03', '1949-04', '1949-05', '1949-06',
             '1949-07', '1949-08', '1949-09', '1949-10',
             ...
             '1959-03', '1959-04', '1959-05', '1959-06', '1959-07', '1959-08',
             '1959-09', '1959-10', '1959-11', '1959-12'],
            dtype='period[M]', name='Month', length=132)

In [131]:
df.head()

,Passengers
Month,
1949-01,112
1949-02,118
1949-03,132
1949-04,129
1949-05,121


In [132]:
predictions = df.copy()
predictions['y_pred']= sarima111010.predict(df.index)
predictions['residuals']= sarima111010.predict_residuals(df[['Passengers']] )
predictions

,Passengers,y_pred,residuals
Month,,,
1949-01,112,NaN,NaN
1949-02,118,112.524395,5.475605
1949-03,132,118.526680,13.473320
1949-04,129,132.525194,-3.525194
1949-05,121,129.526493,-8.526493
...,...,...,...
1960-08,606,629.389618,-23.389618
1960-09,508,533.891130,-25.891130
1960-10,461,478.430591,-17.430591


In [133]:
predictions.dropna(inplace=True)
predictions

,Passengers,y_pred,residuals
Month,,,
1949-02,118,112.524395,5.475605
1949-03,132,118.526680,13.473320
1949-04,129,132.525194,-3.525194
1949-05,121,129.526493,-8.526493
1949-06,135,121.525917,13.474083
...,...,...,...
1960-08,606,629.389618,-23.389618
1960-09,508,533.891130,-25.891130
1960-10,461,478.430591,-17.430591


In [134]:
from sklearn.metrics import r2_score, mean_absolute_percentage_error

In [135]:
r2_score(predictions.Passengers, predictions.y_pred)

0.9866140471646563

In [136]:
mean_absolute_percentage_error(predictions.Passengers, predictions.y_pred)

0.040986395586043306

---
## Predict Model

This function predicts Label using a trained model. When data is None, it predicts label on the holdout set.

note: so far, our best model is the ets model


In [144]:
holdout_pred = exp.predict_model(sarima111010)

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,ARIMA,0.7354,0.8041,22.3941,27.7807,0.0505,0.0485,0.8607


## Finalize Model

This function trains a given estimator on the entire dataset including the holdout set.

Model finalization is the last step in the experiment. This workflow will eventually lead you to the best model for use in making predictions on new and unseen data. The finalize_model() function fits the model onto the complete dataset including the test/hold-out sample. The purpose of this function is to train the model on the complete dataset before it is deployed in production.

In [145]:
final_model = exp.finalize_model(sarima111010)

In [146]:
final_model

ForecastingPipeline(steps=[('forecaster',
                            TransformedTargetForecaster(steps=[('model',
                                                                ARIMA(order=(1,
                                                                             1,
                                                                             1),
                                                                      seasonal_order=(0,
                                                                                      1,
                                                                                      0,
                                                                                      12)))]))])

---
## Final prediciton on unseen data

The predict_model() function is also used to predict on the unseen dataset.

In [147]:
exp.plot_model(plot='train_test_split')

In [149]:
exp.plot_model(final_model, plot='forecast', data_kwargs={'fh':24, 'labels':['SARIMA(1,1,1)(0,1,0,12)']})

In [150]:
unseen_predictions = exp.predict_model(final_model, fh=24)
unseen_predictions

,y_pred
1961-01,444.6179
1961-02,418.6627
1961-03,446.8790
1961-04,489.0439
1961-05,500.2242
1961-06,563.3999
1961-07,650.5770
1961-08,634.7536
1961-09,536.9304
1961-10,490.1072


## Save Model

This function saves the transformation pipeline and trained model object into the current working directory as a pickle file for later use.

In [152]:
exp.save_model(final_model, 'best_arima_model')

Transformation Pipeline and Model Successfully Saved


(ForecastingPipeline(steps=[('forecaster',
                             TransformedTargetForecaster(steps=[('model',
                                                                 ForecastingPipeline(steps=[('forecaster',
                                                                                             TransformedTargetForecaster(steps=[('model',
                                                                                                                                 ARIMA(order=(1,
                                                                                                                                              1,
                                                                                                                                              1),
                                                                                                                                       seasonal_order=(0,
                                                

## Load model

This function loads a previously saved pipeline.



In [153]:
my_model = load_model('best_arima_model')

Transformation Pipeline and Model Successfully Loaded


In [154]:
my_model

ForecastingPipeline(steps=[('forecaster',
                            TransformedTargetForecaster(steps=[('model',
                                                                ForecastingPipeline(steps=[('forecaster',
                                                                                            TransformedTargetForecaster(steps=[('model',
                                                                                                                                ARIMA(order=(1,
                                                                                                                                             1,
                                                                                                                                             1),
                                                                                                                                      seasonal_order=(0,
                                                        

# Done!

In [ ]:
!jupyter nbconvert --to html Module4_ARIMA.ipynb

[NbConvertApp] Converting notebook Module4_ARIMA.ipynb to html
/usr/local/lib/python3.8/dist-packages/nbconvert/filters/datatypefilter.py:39: UserWarning: Your element with mimetype(s) dict_keys(['application/vnd.colab-display-data+json']) is not able to be represented.
  warn("Your element with mimetype(s) {mimetypes}"
[NbConvertApp] Writing 858336 bytes to Module4_ARIMA.html


PyCaret uses a package called "Plotly" to create interactive visualizations, which can be viewed in the Jupyter notebook or in a web browser, but may not be visible in a static HTML format.

When exporting a PyCaret notebook to an HTML format using nbconvert, the Plotly graphs are converted to a static image format by default, which can lead to the loss of interactivity and some visual details.
